In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
import pandas as pd
import time

# 크롬 옵션 설정
options = Options()
options.add_argument('--headless')  # 브라우저 창 안 띄움 (테스트용)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

ModuleNotFoundError: No module named 'selenium'

In [ ]:
# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 날짜 범위 설정
start_date = pd.to_datetime('2023-03-01')
end_date = pd.to_datetime('2024-02-28')

# 결과 저장 리스트
data = []

# User-Agent 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
}

# 날짜 루프
date = start_date
while date <= end_date:
    date_str = date.strftime('%Y%m%d')
    url = f'https://news.naver.com/main/ranking/popularDay.naver?rankingType=popular_day&sectionId=101&date={date_str}'

    try:
        res = requests.get(url, headers=headers, timeout=5)
        if res.status_code != 200:
            print(f"{date_str} 요청 실패: {res.status_code}")
            date += timedelta(days=1)
            continue

        soup = BeautifulSoup(res.text, 'html.parser')
        # 최근 구조에서는 ranking_list ol 태그 안에 있음
        news_list = soup.select('ol.ranking_list li')

        for item in news_list:
            try:
                title = item.select_one('div.ranking_headline').get_text(strip=True)
                data.append({
                    '날짜': date_str,
                    '뉴스제목': title
                })
            except AttributeError:
                continue

    except Exception as e:
        print(f"{date_str} 크롤링 중 오류: {e}")

    date += timedelta(days=1)

# DataFrame으로 변환
df = pd.DataFrame(data)

# 결과 확인
print(df.head())
print(f"총 {len(df)}건의 뉴스가 수집되었습니다.")

# CSV 저장
df.to_csv('naver_news_20230301_20240228.csv', index=False, encoding='utf-8-sig')


Empty DataFrame
Columns: []
Index: []
총 0건의 뉴스가 수집되었습니다.
